# This notebook goes through dataset creation for quantitative models. 
**!!!** If not running on bizon or not using GRCh38 or mm9 genomes please download fasta files and change paths in the set_genome function

**required scripts for running this notebook:**
- /home/shush/profile/QuantPred/datasets/basenji_data.py
- /home/shush/profile/QuantPred/datasets/basenji_data_read.py
- /home/shush/profile/QuantPred/datasets/basenji_data_write.py
- /home/shush/profile/QuantPred/datasets/genome.py
- /home/shush/profile/QuantPred/datasets/util.py
- /home/shush/profile/QuantPred/datasets/dna_io.py

or just pull git@github.com:amberT15/QuantPred.git

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import subprocess
import yaml, os, shutil, sys
import util
import json
from natsort import natsorted
import tensorflow as tf

In [3]:
def set_genome(genome): 
    '''choose genome file paths for chr size, fa and unmappable genome segments (optional)'''
    genome_dict = {'hg38': {'size':  '/home/shush/genomes/GRCh38_EBV.chrom.sizes.tsv',
                              'fa':  '/home/shush/genomes/hg38.fa',
                              'unmap':  '/home/shush/genomes/GRCh38_unmap.bed'},
                  'mm9': {'size':  '/home/shush/genomes/mm9.chrom.sizes',
                          'fa':  '/home/shush/genomes/mm9.fa',
                          'unmap':  '/home/shush/genomes/mm9-blacklist.bed'}}
            
    assert genome in genome_dict.keys(), 'Unknown genome!'
    return genome_dict[genome]

def write_basenji_samplefile(bigwig_filepaths, basenji_samplefile):
    df = pd.DataFrame(columns =['index', 'identifier', 'file', 'sum_stat', 'description'])
    # per file, get the filename
    for b, bigwig_filepath in enumerate(bigwig_filepaths):
        # make entry in basenji samplefile
        df.loc[b] = [b, os.path.basename(bigwig_filepath).split('.b')[0], bigwig_filepath, 'mean', '']

    # write to csv files
    df.to_csv(basenji_samplefile, index=None, sep='\t')
    
def write_basset_samplefile(bed_filepaths, basset_samplefile):
    print('Generating merged samplefile for the entire bedfile set')
    df = pd.DataFrame(columns =['identifier', 'file'])
    # per file, get the filename
    for b, bedfile_path in enumerate(bed_filepaths):
        # make entry in basenji samplefile
        df.loc[b] = [os.path.basename(bedfile_path).split('.b')[0], bedfile_path]
    # write to csv files
    df.to_csv(basset_samplefile, index=None, header=None, sep='\t')
    


The most relevant options here are:

| Argument | Note |
|:---|:---|
| chroms_only | if 'all' creates train, val, test, if specific chromosomes then creates test set from only those|
| input_size| input size of the genomic|
| base_dir | the output directory for the tfr files |
| bigwig_paths_pattern | regexp pattern that will collect all the bigwig files |
| bigwig_filepaths | can set this instead as a list of all the bws if don't want to use glob |
| bedfile_paths_pattern | same for bed files of genomic regions to focus on IF you want peak centered dataset |
| bed_filepaths | --.-- |
| pool_window | bin size, if set to 1 can bin later in the training |
| dilation_rate | fraction of data to include, can set to 0.1 to test pipelines|



In [32]:
!ls /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/A/Bigwig/

BalbC_During_1stPreg_27ac.bigwig  BalbC_Parous_27ac.bigwig
BalbC_During_2ndPreg_27ac.bigwig  BalbC_Virgin_27ac.bigwig


In [33]:
input_size = 3072
base_dir = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC'
bigwig_paths_pattern = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/*/Bigwig/*BalbC*'
bigwig_filepaths = [f for f in glob.glob(bigwig_paths_pattern) if (f.endswith('bigwig') or f.endswith('bw')) 
                    and 'Input' not in f]
# bedfile_paths_pattern = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/*/Bed/*'
# bed_filepaths = [f for f in glob.glob(bedfile_paths_pattern) if f.endswith('bed') or f.endswith('gz')]

# optional arguments
pool_window = 1 
dilation_rate = 1 
valid_chr = 'chr9'
test_chr = 'chr8'
genome = 'mm9'
chroms_only = 'all'
norm = 'none'
threshold = 0 # threshold for train and val
test_threshold = 0 # test threshold
# only change these if you want overlapping inputs
step = 0
stride_test = 1 #e.g. set to 0.333333333333 to move by 1K if 3K input
padding = 'none'
util.make_directory(base_dir) # create dir if not there already
# basset_samplefile = os.path.join(base_dir, 'basset_samplefile.csv') # change to random for thresholded
basset_samplefile = 'random'
basenji_samplefile = os.path.join(base_dir, 'basenji_samplefile.csv')


Making directory: /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC


In [34]:
len(set([os.path.basename(b) for b in bigwig_filepaths]))

4

In [35]:
write_basenji_samplefile(bigwig_filepaths, basenji_samplefile) # write pre-requisite file for the pipeline specifying bw paths

if basset_samplefile != 'random':
    write_basset_samplefile(bed_filepaths, basset_samplefile)# write pre-requisite file for the pipeline specifying bed paths

In [36]:
config_path = os.path.join(base_dir, 'config.yaml')
config = {}

config['genomefile'] = set_genome(genome)

config['chroms']={'valid': valid_chr, 'test': test_chr, 'only': chroms_only}
config['input'] = {'downsample': dilation_rate, 'size':input_size, 'pool':pool_window, 
                   'norm': norm, 'step':step, 'padding':padding}
config['samplefile'] = {'basset': basset_samplefile, 'basenji':basenji_samplefile}

config['threshold'] = threshold
config['test_threshold'] = test_threshold
config['output'] = {'dir': base_dir, 
                   'prefix': 'i_%i_w_%i' % (config['input']['size'], config['input']['pool'])}
config['stride_test'] = stride_test 

In [37]:
with open('config.yaml', 'w') as file:
    documents = yaml.dump(config, file, default_flow_style=False)

In [38]:
! ./bw_to_tfr.sh

!!!
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC
random
Chopping randomly
Running basenji data processing
/home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/basenji_samplefile.csv -g /home/shush/genomes/mm9-blacklist.bed -l 3072 -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 -t chr8 -v chr9 -w 1 --local -d 1 --norm none --step 0 --padding none -p 19 --threshold 0 --test_threshold 0 --only_chroms all --stride_test 1
['/home/shush/genomes/mm9.fa', '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/basenji_samplefile.csv']
Using test set threshold of 0.0
stride_train 1 converted to 3072.000000
stride_test 1 converted to 3072.000000
CHR
Contigs divided into
 Train:  2816 contigs, 2209062064 nt (0.8964)
 Valid:   114 contigs,  123797972 nt (0.0502)
 Test:     18 contigs,  131470271 nt (0.0533)
<generator object divide_contigs_chr.<locals>.<genexpr> at 0x7f5001f31228>
./basenji

./basenji_data_write.py -s 3328 -e 3584 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-13.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 3584 -e 3840 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-14.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000

./basenji_data_write.py -s 7680 -e 7936 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-30.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 7936 -e 8192 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-31.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000

./basenji_data_write.py -s 11776 -e 12032 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-46.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 12032 -e 12288 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-47.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0

./basenji_data_write.py -s 14592 -e 14848 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-57.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 14848 -e 15104 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/train-58.tfr train -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0

...

./basenji_data_write.py -s 745582 -e 745838 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-109.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 745838 -e 746094 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-110.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_thres

./basenji_data_write.py -s 748398 -e 748654 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-120.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 748654 -e 748910 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-121.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_thres

*
*
*
*
*
250
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
251
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
252
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
246
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
249
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
254
*
*
*
*
*
*
*
*
*
*
250
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
./basenji_

./basenji_data_write.py -s 755566 -e 755822 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-148.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_threshold 0.000000
./basenji_data_write.py -s 755822 -e 756078 --umap_clip 1.000000 /home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/sequences.bed /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/seqs_cov /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/tfrecords/valid-149.tfr valid -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1 --threshold 0.000000 --test_thres

*
*
*
*
*
252
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
46
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
*
*
*
*
*
40529
*
*
*
*
*
./basenji_data_write.py:232: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  values = values.flatten().tostring()
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_datasets/BalbC/i_3072_w_1/statistics.json
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~
~~~


In [30]:
! head /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/basenji_samplefile.csv

BalbC_Virgin_27ac_q1e-03	/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/A/Bed/BalbC_Virgin_27ac_q1e-03.bed
BalbC_During_2ndPreg_27ac_q1e-03	/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/A/Bed/BalbC_During_2ndPreg_27ac_q1e-03.bed
BalbC_During_1stPreg_27ac_q1e-03	/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/A/Bed/BalbC_During_1stPreg_27ac_q1e-03.bed
BalbC_Parous_27a_q1e-03	/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/A/Bed/BalbC_Parous_27a_q1e-03.bed


In [85]:
# sanity check for dataset splits
! cut -f4 /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/i_3072_w_1/sequences.bed | sort | uniq -c

  42787 test
 717678 train
  40239 valid


In [86]:
# summary of the configs corresponding to the new dataset
! cat /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/i_3072_w_1/config.yaml

chroms:
  only: all
  test: chr8
  valid: chr9
genomefile:
  fa: /home/shush/genomes/mm9.fa
  size: /home/shush/genomes/mm9.chrom.sizes
  unmap: /home/shush/genomes/mm9-blacklist.bed
input:
  downsample: 1
  norm: none
  padding: none
  pool: 1
  size: 3072
  step: 0
output:
  dir: /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset
  prefix: i_3072_w_1
samplefile:
  basenji: /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/basenji_samplefile.csv
  basset: random
stride_test: 1
test_threshold: -1
threshold: 2


The data is now in tfrecords which is optimized for faster i/o and speeds up training. But if needed it can easily be converted to numpy arrays. Here I look at the test set only and retireve the sequences, targets and the coordinates as well (which is why we need `coords=True, number_data_types=3`). 

In [108]:
data_dir = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/tfr_dataset/i_3072_w_1/'
sts = load_stats(data_dir)
testset = make_dataset(data_dir, 'test', sts, batch_size=32, shuffle=False, coords=True)
targets = pd.read_csv(data_dir+'targets.txt', sep='\t')['identifier'].values

In [109]:
targets

array(['CAGMYC_DD5_Parous_Input_27ac', 'CAGMYC_DD5_Virgin_Input_27ac',
       'CAGMYC_DD5_Parous_Rep1_ATAC', 'CAGMYC_DD5_Virgin_Rep1_ATAC',
       'CAGMYC_DD5_Parous_Rep2_ATAC', 'CAGMYC_DD5_Virgin_27ac',
       'CAGMYC_DD5_Parous_27ac', 'CAGMYC_DD5_Virgin_Rep2_ATAC',
       'CAGMYC_DD5_Virgin_Input_27ac', 'CAGMYC_DD2_Virgin_27ac',
       'CAGMYC_DD2_Virgin_Input_27ac', 'CAGMYC_DD5_Virgin_27ac',
       'BalbC_Virgin_27ac', 'BalbC_Parous_27ac',
       'BalbC_During_2ndPreg_27ac', 'BalbC_During_1stPreg_27ac'],
      dtype=object)

In [110]:
testset

<DatasetV1Adapter shapes: ((None,), (None, 3072, 4), (None, 3072, 16)), types: (tf.string, tf.float32, tf.float32)>

In [118]:
C, X, Y = convert_tfr_to_np(testset, number_data_types=3)

In [123]:
print(C[1000])

b'chr8_3072000_3075072'


# Make just the test set with a 2K window size for evaluation

this is useful if you already have a dataset but want to make a smaller one only from specific chromosomes and potentially with other thresholds, peak files, etc.

In [74]:
input_size = 2048
base_dir = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8'
bigwig_paths_pattern = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/*/Bigwig/*'
bigwig_filepaths = [f for f in glob(bigwig_paths_pattern) if f.endswith('bigwig') or f.endswith('bw')]
bedfile_paths_pattern = '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/datasets/*/Bed/*'
bed_filepaths = [f for f in glob(bedfile_paths_pattern) if f.endswith('bed') or f.endswith('gz')]

# optional arguments
pool_window = 1 
dilation_rate =0.1 
valid_chr = 'chr9'
test_chr = 'chr8'
genome = 'mm9'
chroms_only = 'chr8' #*********** <------ ********* change this to comma separated list of chr you want to include
norm = 'none'
threshold = 0 # threshold for train and val
test_threshold = 0 # test threshold
# only change these if you want overlapping inputs
step = 0
stride_test = 1 #e.g. set to 0.333333333333 to move by 1K if 3K input
padding = 'none'
util.make_directory(base_dir) # create dir if not there already
basset_samplefile =  'random' #os.path.join(base_dir, 'basset_samplefile.csv') 
basenji_samplefile = os.path.join(base_dir, 'basenji_samplefile.csv')



Making directory: /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8


In [75]:

write_basenji_samplefile(bigwig_filepaths, basenji_samplefile) # write pre-requisite file for the pipeline specifying bw paths

if basset_samplefile != 'random':
    write_basset_samplefile(bed_filepaths, basset_samplefile)# write pre-requisite file for the pipeline specifying bed paths

In [76]:

config_path = os.path.join(base_dir, 'config.yaml')
config = {}

In [77]:
config_path = os.path.join(base_dir, 'config.yaml')
config = {}

config['genomefile'] = set_genome(genome)

config['chroms']={'valid': valid_chr, 'test': test_chr, 'only': chroms_only}
config['input'] = {'downsample': dilation_rate, 'size':input_size, 'pool':pool_window, 
                   'norm': norm, 'step':step, 'padding':padding}
config['samplefile'] = {'basset': basset_samplefile, 'basenji':basenji_samplefile}

config['threshold'] = threshold
config['test_threshold'] = test_threshold
config['output'] = {'dir': base_dir, 
                   'prefix': 'i_%i_w_%i' % (config['input']['size'], config['input']['pool'])}
config['stride_test'] = stride_test 

In [78]:
with open('config.yaml', 'w') as file:
    documents = yaml.dump(config, file, default_flow_style=False)

In [79]:
! ./bw_to_tfr.sh

!!!
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8
random
Chopping randomly
Running basenji data processing
/home/shush/genomes/mm9.fa /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8/basenji_samplefile.csv -g /home/shush/genomes/mm9-blacklist.bed -l 2048 -o /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8/i_2048_w_1 -t chr8 -v chr9 -w 1 --local -d 0.1 --norm none --step 0 --padding none -p 19 --threshold 0 --test_threshold 0 --only_chroms chr8 --stride_test 1
['/home/shush/genomes/mm9.fa', '/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8/basenji_samplefile.csv']
Using test set threshold of 0.0
stride_train 1 converted to 2048.000000
stride_test 1 converted to 2048.000000
chr8
CHR
Contigs divided into
 Train:     0 contigs,          0 nt (0.0000)
 Valid:     0 contigs,          0 nt (0.0000)
 Test:     18 contigs,  131470271 nt (1.0000)
<generator object divide_contigs_chr.<locals>.<genexpr> at 0x7fdadafe9228>
./basenji_data_read.py --crop 0 -w 1 -u mean -s 1.000000 

In [80]:
! cut -f4 /mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/colab/chr8/i_2048_w_1/sequences.bed | sort | uniq -c

   6418 test
